# Libraries

In [1]:
import  pandas as pd
import  numpy as np
import  plotly.express as px
import  matplotlib.pyplot as plt
import  plotly.graph_objs as go

from    sklearn.feature_selection import RFE
from    sklearn.ensemble import RandomForestClassifier
from    sklearn.impute import SimpleImputer
from    sklearn.preprocessing import StandardScaler
from    imblearn.over_sampling import SMOTE
from    sklearn.feature_selection import SelectFromModel

In [2]:
RANDOM_SEED = 0

In [3]:
cd ..\

c:\Users\victo\Downloads\00. Estudo Data Science\aps_failure_truck_prediction


c:\Users\victo\Downloads\00. Estudo Data Science\aps_failure_truck_prediction\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Functions

In [70]:
def get_zero_variance_features(dataframe, target_name: str):
    """
    Receive a dataframe and the target name
    Calculates the variance for each feature
    Return a list with the zero variance features
    """
    columns_to_drop = []
    X = dataframe.drop(columns=target_name)
    for i in X:
        if X[i].std() == 0:
            columns_to_drop.append(i)
            print(f'Column {i} has 0 variance and can be excluded')
    return columns_to_drop

# Train

## Load data

In [71]:
df_air_sys_prev = pd.read_csv('data\\original_data\\air_system_previous_years.csv',
                              encoding='utf-8',
                              na_values=['na']
                              )

# Change columns type to float
columns_object_to_float = df_air_sys_prev.drop(columns='class').columns
df_air_sys_prev[columns_object_to_float] = df_air_sys_prev[columns_object_to_float].astype('float64')

# Change class to int dummies
map_class = {'neg': 0, 'pos': 1}
df_air_sys_prev['class'] = df_air_sys_prev['class'].map(map_class)

df_air_sys_prev.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,0,76698.0,NaN,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,0,33058.0,NaN,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,0,41040.0,NaN,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,0,12.0,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,0,60874.0,NaN,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [6]:
df_air_sys_prev.dtypes

class       int64
aa_000    float64
ab_000    float64
ac_000    float64
ad_000    float64
           ...   
ee_007    float64
ee_008    float64
ee_009    float64
ef_000    float64
eg_000    float64
Length: 171, dtype: object

In [7]:
df_air_sys_prev.isna().sum()

class         0
aa_000        0
ab_000    46329
ac_000     3335
ad_000    14861
          ...  
ee_007      671
ee_008      671
ee_009      671
ef_000     2724
eg_000     2723
Length: 171, dtype: int64

In [8]:
# List of columns with NaN values
columns_with_nan = df_air_sys_prev.isna().sum()[df_air_sys_prev.isna().sum() > 0].index
columns_with_nan

Index(['ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000', 'ag_001',
       'ag_002', 'ag_003', 'ag_004',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=169)

In [53]:
# Use the median to replace NaN values
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer.fit(df_air_sys_prev[columns_with_nan])

SimpleImputer(strategy='median')

In [54]:
# Replace Nan values of each column with the relative median
df_air_sys_prev[columns_with_nan] = imputer.transform(df_air_sys_prev[columns_with_nan])
df_air_sys_prev.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,0,76698.0,0.0,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,0,33058.0,0.0,0.000000e+00,126.0,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,0,41040.0,0.0,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,0,12.0,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,0,60874.0,0.0,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [11]:
df_air_sys_prev['class'].value_counts()

class
0    59000
1     1000
Name: count, dtype: int64

In [12]:
X = df_air_sys_prev.drop(columns=['class'])
y = df_air_sys_prev[['class']]

# Make the balance of the minority class
oversample = SMOTE(sampling_strategy='minority', random_state=RANDOM_SEED)
X_balanced, y_balanced = oversample.fit_resample(X, y)

# Dataframe with balanced data
df_air_sys_prev_balanced = y_balanced.join(X_balanced)
df_air_sys_prev_balanced.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,0,76698.0,0.0,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,0,33058.0,0.0,0.000000e+00,126.0,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,0,41040.0,0.0,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,0,12.0,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,0,60874.0,0.0,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [13]:
df_air_sys_prev_balanced['class'].value_counts()

class
0    59000
1    59000
Name: count, dtype: int64

In [14]:
# Remove zero variance features
columns_to_drop = get_zero_variance_features(df_air_sys_prev_balanced, target_name='class')
df_air_sys_prev_zero_std = df_air_sys_prev_balanced.drop(columns=columns_to_drop).copy()
df_air_sys_prev_zero_std.head()

Column cd_000 has 0 variance and can be excluded


,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,0,76698.0,0.0,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,0,33058.0,0.0,0.000000e+00,126.0,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,0,41040.0,0.0,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,0,12.0,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,0,60874.0,0.0,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [15]:
# # Save train dataframe post-processing
# df_air_sys_prev_zero_std.to_csv('data\processed_data\df_train.csv',
#                                 encoding='utf-8',
#                                 sep=',',
#                                 index=False)

In [16]:
X_train = df_air_sys_prev_zero_std.drop(columns='class')
y_train = df_air_sys_prev_zero_std['class']

In [17]:
# Calculate the feature importance from Random Forest
feature_selector_rand_forest = SelectFromModel(estimator=RandomForestClassifier(random_state=RANDOM_SEED)).fit(X_train, y_train)

In [18]:
df_feature_importance = pd.DataFrame({'feature_importance': feature_selector_rand_forest.estimator_.feature_importances_},
                                      index=feature_selector_rand_forest.estimator_.feature_names_in_
                                      )
df_feature_importance.sort_values('feature_importance',
                                   ascending=False,
                                   inplace=True
                                   )
df_feature_importance

,feature_importance
ci_000,1.010276e-01
aa_000,8.550138e-02
dn_000,5.413101e-02
aq_000,4.108511e-02
ck_000,4.007663e-02
...,...
eg_000,2.404758e-05
as_000,2.343172e-05
dm_000,2.063074e-06
dl_000,1.025552e-06


In [19]:
# Quantile 90%
q90 = df_feature_importance['feature_importance'].quantile(0.90)

# Dataframe with values over quantile 90%
df_feature_importance_over_q90 = df_feature_importance[df_feature_importance['feature_importance'] > q90].copy()
df_feature_importance_over_q90

,feature_importance
ci_000,0.101028
aa_000,0.085501
dn_000,0.054131
aq_000,0.041085
ck_000,0.040077
bv_000,0.037443
ah_000,0.036728
bu_000,0.036466
bh_000,0.032666
ap_000,0.032044


In [20]:
# Total features over quantile 90%
df_feature_importance_over_q90.shape[0]

17

In [21]:
# Plot features importance
fig = px.bar(y=df_feature_importance_over_q90['feature_importance'],
             x=df_feature_importance_over_q90.index,
             text_auto=True
             )
fig.update_traces(textfont=dict(size=14))
fig.update_layout(height=400,
                  width=1200,
                  showlegend=False,
                  margin=dict(l=10, r=10, b=10, t=10, pad=0),
                  xaxis=dict(title='Feature', tickfont=dict(size=14)),
                  yaxis=dict(title='Feature importance',
                             titlefont=dict(size=14),
                             dtick=0.01
                             )
                  )
fig.show()

## Graphical analysis

In [22]:
# Concatenate class with important features
df_air_sys_prev_important_features = df_air_sys_prev_zero_std[['class'] + df_feature_importance_over_q90.index.to_list()]
df_air_sys_prev_important_features.head()

,class,ci_000,aa_000,dn_000,aq_000,ck_000,bv_000,ah_000,bu_000,bh_000,ap_000,bg_000,bb_000,bt_000,ag_004,ee_000,an_000,bj_000
0,0,5245752.00,76698.0,62282.0,1132040.0,916567.68,6700214.0,2551696.0,6700214.0,97518.0,1766008.0,2551696.0,6700214.0,76698.08,37250.0,965866.0,4933296.0,799478.0
1,0,2291079.36,33058.0,33736.0,338544.0,643536.96,3646660.0,1393352.0,3646660.0,49028.0,1084598.0,1393352.0,3646660.0,33057.51,18254.0,664504.0,2560898.0,392208.0
2,0,2322692.16,41040.0,13876.0,153698.0,236099.52,2673338.0,1234132.0,2673338.0,28804.0,300796.0,1234132.0,2673338.0,41040.08,1648.0,262032.0,2371990.0,139730.0
3,0,2135.04,12.0,232.0,1014.0,4525.44,21614.0,2668.0,21614.0,184.0,10764.0,2668.0,21614.0,12.69,2212.0,5670.0,10184.0,3090.0
4,0,3565684.80,60874.0,44946.0,551022.0,379111.68,4289260.0,1974038.0,4289260.0,86454.0,1058136.0,1974038.0,4289260.0,60874.03,43752.0,404740.0,3230626.0,399410.0


In [23]:
# # Calculate and plot features correlations
# corr_air_sys_prev_balanced = df_air_sys_prev_important_features.drop(columns='class').corr()

# fig = px.imshow(corr_air_sys_prev_balanced,
#                 text_auto=True,
#                 color_continuous_scale='Viridis_r',
#                 aspect='auto'
#                 )
# fig.update_layout(height=1000,
#                   width=1000,
#                   showlegend=True,
#                   margin=dict(l=30, r=10, b=10, t=30, pad=0),
#                   yaxis=dict(ticksuffix = "  "),
#                   xaxis=dict(ticksuffix = "  ")
#                   )
# fig.show()

In [24]:
# # Plot the features correlations by class
# fig = px.scatter_matrix(df_air_sys_prev_important_features, color='class',
#                         color_continuous_scale='Viridis_r'
#                         )
# fig.update_traces(diagonal_visible=False)
# fig.update_layout(height=800,
#                   width=800,
#                   showlegend=False,
#                   margin=dict(l=10, r=10, b=10, t=10)
#                   )
# fig.show()

In [25]:
# # Save train dataframe with important features only
# df_air_sys_prev_important_features.to_csv('data\processed_data\df_train_important_features.csv',
#                                           encoding='utf-8',
#                                           sep=',',
#                                           index=False
#                                           )

# Test

In [49]:
df_air_sys_present = pd.read_csv('data\\original_data\\air_system_present_year.csv',
                                 encoding='utf-8',
                                 na_values=['na']
                                 )

# Change columns type to float
columns_object_to_float = df_air_sys_present.drop(columns='class').columns
df_air_sys_present[columns_object_to_float] = df_air_sys_present[columns_object_to_float].astype('float64')

# Change class to int dummies
map_class = {'neg': 0, 'pos': 1}
df_air_sys_present['class'] = df_air_sys_present['class'].map(map_class)
df_air_sys_present.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,0,60.0,0.0,20.0,12.0,0.0,0.0,0.0,0.0,0.0,...,1098.0,138.0,412.0,654.0,78.0,88.0,0.0,0.0,0.0,0.0
1,0,82.0,0.0,68.0,40.0,0.0,0.0,0.0,0.0,0.0,...,1068.0,276.0,1620.0,116.0,86.0,462.0,0.0,0.0,0.0,0.0
2,0,66002.0,2.0,212.0,112.0,0.0,0.0,0.0,0.0,0.0,...,495076.0,380368.0,440134.0,269556.0,1315022.0,153680.0,516.0,0.0,0.0,0.0
3,0,59816.0,NaN,1010.0,936.0,0.0,0.0,0.0,0.0,0.0,...,540820.0,243270.0,483302.0,485332.0,431376.0,210074.0,281662.0,3232.0,0.0,0.0
4,0,1814.0,NaN,156.0,140.0,0.0,0.0,0.0,0.0,0.0,...,7646.0,4144.0,18466.0,49782.0,3176.0,482.0,76.0,0.0,0.0,0.0


In [52]:
df_air_sys_present.dtypes

class       int64
aa_000    float64
ab_000    float64
ac_000    float64
ad_000    float64
           ...   
ee_007    float64
ee_008    float64
ee_009    float64
ef_000    float64
eg_000    float64
Length: 171, dtype: object

In [51]:
df_air_sys_present.isna().sum()

class         0
aa_000        0
ab_000    12363
ac_000      926
ad_000     3981
          ...  
ee_007      192
ee_008      192
ee_009      192
ef_000      762
eg_000      762
Length: 171, dtype: int64

In [55]:
# Replace Nan values of each column with the relative median
df_air_sys_present[columns_with_nan] = imputer.transform(df_air_sys_present[columns_with_nan])
df_air_sys_present.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,0,60.0,0.0,20.0,12.0,0.0,0.0,0.0,0.0,0.0,...,1098.0,138.0,412.0,654.0,78.0,88.0,0.0,0.0,0.0,0.0
1,0,82.0,0.0,68.0,40.0,0.0,0.0,0.0,0.0,0.0,...,1068.0,276.0,1620.0,116.0,86.0,462.0,0.0,0.0,0.0,0.0
2,0,66002.0,2.0,212.0,112.0,0.0,0.0,0.0,0.0,0.0,...,495076.0,380368.0,440134.0,269556.0,1315022.0,153680.0,516.0,0.0,0.0,0.0
3,0,59816.0,0.0,1010.0,936.0,0.0,0.0,0.0,0.0,0.0,...,540820.0,243270.0,483302.0,485332.0,431376.0,210074.0,281662.0,3232.0,0.0,0.0
4,0,1814.0,0.0,156.0,140.0,0.0,0.0,0.0,0.0,0.0,...,7646.0,4144.0,18466.0,49782.0,3176.0,482.0,76.0,0.0,0.0,0.0


In [57]:
df_air_sys_present.isna().sum().sum()

np.int64(0)

In [69]:
df_air_sys_present.to_csv('data\processed_data\df_test.csv',
                          encoding='utf-8',
                          sep=',',
                          index=False)

In [61]:
import  os

In [65]:
os.path.join('..', 'data', 'prediction') + '\\prediction'

'..\\data\\prediction\\prediction'

In [68]:
os.path.join('..', 'artifacts', 'df_train_scaled.pkl')

'..\\artifacts\\df_train_scaled.pkl'